In [2]:
import numpy as np  # type: ignore
import pandas as pd  # type: ignore

import os

import matplotlib.pyplot as plt  # type: ignore
import subprocess

import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats
from rasterio.warp import calculate_default_transform, reproject, Resampling

%matplotlib inline

In [ ]:
path = r"data\EU_SoilHydroGrids_250m_2017Feb08"

grid_cells = {}

files = []
folders = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for folder in d:
        folders.append(folder)
        
        for r1, d1, f1 in os.walk(os.path.join(path, folder)):
        
            for file in f1:
                if f"FC_M_sl1_{folder}.tif" in file:
                    files.append(os.path.join(r1, file))
                    data_src = rasterio.open(os.path.join(r1, file))
                    srs = data_src.crs
                    bounds = data_src.bounds
                    sub_obj = {
                        'grid_id': folder,
                        'srs': srs,
                        'bounds': bounds
                    }
                    grid_cells.update({ folder: sub_obj})
                    

for folder in folders:
    print(folder)

for file in files:
    print(file)

display(grid_cells)

In [ ]:
import fiona
from shapely.geometry import box, Polygon, shape, mapping
from collections import OrderedDict

schema = {
    "geometry": "Polygon",
    "properties": OrderedDict([
        ("grid_id", "str:200"),
    ])
}

def make_poly_from_bounds(bounds):
    # (minx, miny, maxx, maxy) or a Polygon instance
    bbox = box(bounds.left, bounds.bottom, bounds.right, bounds.top)
    return bbox

def make_feature_from_dict(grid_obj):
    return {
        "geometry": mapping(make_poly_from_bounds(grid_obj.get('bounds'))),
        "properties": {"grid_id": grid_obj.get('grid_id')}
    }
    
features = []

for obj in grid_cells.values():
    # display(obj)
    feat = make_feature_from_dict(obj)
    features.append(feat)

crs_wkt = data_src.crs.wkt

# From https://doi.org/10.5281/zenodo.3446747 Grid tiles for identifying EU-SoilHydroGrids tiles
with fiona.open(r"data\EU_SoilHydroGrids_250m_2017Feb08\grid_cells.shp", "w", driver="ESRI Shapefile", schema=schema, crs_wkt=crs_wkt) as collection:
    collection.writerecords(features)
    print(len(collection))
    collection.flush()

In [ ]:


path_eesti_250m = r"data\Estonia_EU_Hydrosoilgrids\Estonia_250m"

for r, d, f in os.walk(path_eesti_250m):
    for folder in d:
        for n in range(1,8):
            try:
                # We handle the connections with "with"
                with rasterio.open(os.path.join(r, folder, f"FC_M_sl{n}_{folder}.tif")) as src:
                    FC = src.read(1, masked=True)
                    nodataval = src.nodata

                with rasterio.open(os.path.join(r, folder, f"WP_M_sl{n}_{folder}.tif")) as src:
                    WP = src.read(1, masked=True)

                # Allow division by zero
                np.seterr(divide='ignore', invalid='ignore')

                # Calculate NDVI
                AWC = FC.astype(np.uint8) - WP.astype(np.uint8)
                
                # write_out_awc_raster(folder, n)
                # Define spatial characteristics of output object (basically they are analog to the input)
                kwargs = src.meta

                # Update kwargs (change in data type)
                kwargs.update(dtype=rasterio.uint8, count = 1, nodata = nodataval, masked = True)

                # Let's see what is in there
                print(kwargs)

                with rasterio.open(os.path.join(r, folder, f"AWC_M_sl{n}_{folder}.tif"), 'w', **kwargs) as dst:
                    dst.write_band(1, AWC.astype(rasterio.uint8))
                print(subprocess.check_output("gdalinfo " + os.path.join(r, folder, f"AWC_M_sl{n}_{folder}.tif"), shell=True))
                        
            except Exception as ex:
                print(ex)


## Now loading the soil polygons

- actually, load the big soil shp, only keep some columns and write out as shp again
- then run the rasterstats with keeping the geojson true and orig_fid attribute in particular
- then build a dataframe out of it and start developing the magic for the different layer aggregations
- once the AWC is mean per SOL_N layers and depth based on the original measured AWC EU hydrogrid depths join this AWC back to the main soil db (and basically only overwrite the SOL_AWC_1-4)

In [4]:
import numpy as np  # type: ignore
import pandas as pd  # type: ignore

import fiona  # type: ignore
from fiona.crs import from_epsg # type: ignore
import geopandas as gpd  # type: ignore

eesti_soil_red1_validatesoil = gpd.read_file("../data_deposit/EstSoil-EH_v1.0.shp", encoding='utf-8')


columns_to_keep = [ "orig_fid",
                    "nlayers",
                    "SOL_ZMX",
                    "SOL_Z1",
                    "SOL_Z2",
                    "SOL_Z3",
                    "SOL_Z4",
                   'SOL_CLAY1',
                   'SOL_SILT1',
                   'SOL_SAND1',
                   'SOL_ROCK1',
                    'SOL_BD1',
                   'SOL_SOC1',
                    "geometry"]

eesti_soil_red1_validatesoil_short = eesti_soil_red1_validatesoil[columns_to_keep].copy()
del(eesti_soil_red1_validatesoil)

eesti_soil_red1_validatesoil_short.to_file('../data_deposit/EstSoil-EH_sand_silt_coarse_tmp.shp', encoding='utf-8')



In [8]:
eesti_soil_red1_validatesoil_short.describe()

,orig_fid,nlayers,SOL_ZMX,SOL_Z1,SOL_Z2,SOL_Z3,SOL_Z4
count,745442.000000,745442.000000,745442.000000,745442.000000,745442.000000,745442.000000,745442.000000
mean,372731.913556,1.326452,1021.114909,873.716802,300.013636,22.608660,0.086492
std,215201.422727,0.514279,123.607793,278.752738,457.989242,151.503151,9.767806
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,186360.250000,1.000000,1000.000000,700.000000,0.000000,0.000000,0.000000
50%,372720.500000,1.000000,1000.000000,1000.000000,0.000000,0.000000,0.000000
75%,559104.750000,2.000000,1000.000000,1000.000000,1000.000000,0.000000,0.000000
max,745597.000000,4.000000,4100.000000,4100.000000,3000.000000,3000.000000,2450.000000


In [7]:
import numpy as np  # type: ignore
import pandas as pd  # type: ignore

import functools
import statistics
import operator

import fiona  # type: ignore
from fiona.crs import from_epsg # type: ignore
import geopandas as gpd  # type: ignore

import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats
from rasterio.warp import calculate_default_transform, reproject, Resampling

comp_l1 = {
    'sand': 'soil250_grid_sand_sd1_3301.tif',
    'silt': 'soil250_grid_silt_sd1_3301.tif',
    'clay': 'soil250_grid_clay_sd1_3301.tif',
    'rock': 'soil250_grid_coarsefrag_sd1_3301.tif',
    'bd': 'soil250_grid_bulkdens_sd1_3301.tif',
    'soc': 'soil250_grid_soc_sd1_3301.tif'
}

with fiona.open('../data_deposit/EstSoil-EH_sand_silt_coarse_tmp.shp') as vector_src:
    
    src_crs = vector_src.crs
    display(src_crs)
    src_schema = vector_src.schema
    display(src_schema)

    src_schema['properties']["mean"] = "float"
    src_schema['properties']["std"] = "float"
    
    # for layer in range(1,8):
    for layer in comp_l1.keys():
        
        outputs = zonal_stats(vector_src,
                # f"C:\\dev\\05_geodata\\soil\\soilgrids_download\\soil250_grid_sand_sd{layer}_3301.tif",
                f"C:\\dev\\05_geodata\\soil\\soilgrids_download\\{comp_l1[layer]}",
                stats="mean std",
                all_touched=True, geojson_out=True)
    
        with fiona.open(f"../data_deposit/EstSoil-EH_{layer}_zonal_layer.shp", "w", driver="ESRI Shapefile", schema=src_schema, crs=src_crs) as collection:
            collection.writerecords(outputs)
            print(len(collection))
            collection.flush()


{'init': 'epsg:3301'}

{'properties': OrderedDict([('orig_fid', 'int:18'),
              ('nlayers', 'int:18'),
              ('SOL_ZMX', 'int:18'),
              ('SOL_Z1', 'int:18'),
              ('SOL_Z2', 'float:24.15'),
              ('SOL_Z3', 'float:24.15'),
              ('SOL_Z4', 'float:24.15')]),
 'geometry': 'Polygon'}

745442
745442
745442
745442
745442
745442


In [ ]:
# load geodataframes
for layer in range(1,8):
    next_layer = gpd.read_file(f"../data_deposit/EstSoil-EH_sand_silt_coarse_tmp_zonal_layer_{layer}.shp", encoding='utf-8')
    display(next_layer.isnull().sum())
    display(next_layer.sample(10))
    display(next_layer.dtypes)
    fig, ax = plt.subplots()
    fig = next_layer["mean"].hist(ax=ax)
    plt.show()
    fig, ax = plt.subplots()
    fig = next_layer["mean"].plot(ax=ax)
    plt.show()
    fig, ax = plt.subplots()
    fig = next_layer["std"].hist(ax=ax)
    plt.show()
    fig, ax = plt.subplots()
    fig = next_layer["std"].plot(ax=ax)
    plt.show()
    # next_layer["mean"] = next_layer.isnull().apply(lambda x: x["majority"])
    # display(next_layer.isnull().sum())
    # because to save memory space, the factions after comma were stored as Byte / UInt8
    # next_layer['mean'] = next_layer['mean'] / 100.0
    next_layer.to_file(f"../data_deposit/EstSoil-EH_sand_silt_coarse_tmp_zonal_layer_{layer}.shp", encoding='utf-8')
    
# do_layer_avg_1_4

In [5]:
import numpy as np  # type: ignore
import pandas as pd  # type: ignore

import fiona  # type: ignore
from fiona.crs import from_epsg # type: ignore
import geopandas as gpd  # type: ignore

comp_l1 = {
    'sand': 'soil250_grid_sand_sd1_3301.tif',
    'silt': 'soil250_grid_silt_sd1_3301.tif',
    'clay': 'soil250_grid_clay_sd1_3301.tif',
    'rock': 'soil250_grid_coarsefrag_sd1_3301.tif',
    'bd': 'soil250_grid_bulkdens_sd1_3301.tif',
    'soc': 'soil250_grid_soc_sd1_3301.tif'
}


In [11]:
# layer1 = gpd.read_file("data/eesti_soil_red1_fix_geo_awc_zonal_layer_1.shp", encoding='utf-8')
# layer1.drop(columns=['median','std'], inplace=True)
# layer1.rename(columns={"mean" : "AWC_L1"}, inplace=True)

is_first = True
layer1 = eesti_soil_red1_validatesoil_short

# for layer in range(2,8):
for layer in comp_l1.keys():
    next_layer = gpd.read_file(f"../data_deposit/EstSoil-EH_{layer}_zonal_layer.shp", encoding='utf-8')
    next_layer.rename( columns = {
        "mean" : "mean_1_" + str(layer),
        "std" : "std_1_" + str(layer)},
                      inplace=True)
    next_layer.drop( columns = [
                            "nlayers",
                            "SOL_ZMX",
                            "SOL_Z1",
                            "SOL_Z2",
                            "SOL_Z3",
                            "SOL_Z4",
                            "geometry"], inplace=True)
    layer1 = pd.merge(left=layer1, right=next_layer, left_on='orig_fid', right_on='orig_fid', how='left')

display(layer1.sample(10))
display(layer1.dtypes)

,orig_fid,nlayers,SOL_ZMX,SOL_Z1,SOL_Z2,SOL_Z3,SOL_Z4,geometry,mean_1_sand,std_1_sand,mean_1_silt,std_1_silt,mean_1_clay,std_1_clay,mean_1_rock,std_1_rock,mean_1_bd,std_1_bd,mean_1_soc,std_1_soc
181061,181061,2,1000,450,1000.0,0.0,0.0,"POLYGON ((550896.200 6542570.550, 550907.050 6...",51.142857,2.356060,34.428571,2.258770,14.285714,0.451754,8.571429,0.494872,972.714286,10.898418,35.714286,1.277753
23180,23180,1,1000,1000,0.0,0.0,0.0,"POLYGON ((677905.910 6399573.620, 677908.130 6...",52.000000,0.000000,36.666667,0.471405,11.333333,0.471405,9.000000,0.000000,824.333333,0.942809,37.666667,0.471405
385692,385692,1,1000,1000,0.0,0.0,0.0,"POLYGON ((626929.830 6564780.880, 626924.289 6...",50.800000,1.600000,34.800000,0.748331,14.200000,1.600000,7.600000,0.489898,882.000000,17.742604,40.000000,1.095445
38892,38887,1,1000,1000,0.0,0.0,0.0,"POLYGON ((647461.420 6397865.000, 647458.370 6...",67.000000,0.000000,26.000000,0.000000,7.000000,0.000000,9.000000,0.000000,776.000000,0.000000,53.000000,0.000000
150725,150725,1,1000,1000,0.0,0.0,0.0,"POLYGON ((553524.373 6506828.474, 553514.090 6...",54.000000,0.000000,31.000000,0.000000,15.000000,0.000000,9.000000,0.000000,853.000000,0.000000,48.000000,0.000000
547051,547040,1,1000,1000,0.0,0.0,0.0,"POLYGON ((686183.830 6385638.910, 686180.560 6...",57.500000,0.500000,31.500000,1.500000,11.000000,1.000000,6.500000,0.500000,863.500000,6.500000,41.500000,0.500000
299223,299207,1,1000,1000,0.0,0.0,0.0,"POLYGON ((651212.918 6379668.736, 651208.530 6...",58.500000,0.500000,31.000000,0.000000,10.500000,0.500000,5.000000,0.000000,827.000000,9.000000,41.500000,0.500000
467376,479184,1,1000,1000,0.0,0.0,0.0,"POLYGON ((599154.360 6471782.061, 599151.780 6...",53.333333,0.471405,36.333333,0.942809,11.000000,0.000000,6.666667,0.471405,928.666667,5.185450,37.000000,0.000000
725547,725536,3,1000,400,900.0,1000.0,0.0,"POLYGON ((724143.660 6417291.810, 724145.440 6...",71.000000,2.000000,22.888889,1.523479,5.666667,0.666667,6.333333,0.666667,884.555556,12.093259,42.555556,0.955814
634503,634357,2,1000,700,1000.0,0.0,0.0,"POLYGON ((605957.910 6457171.450, 605948.840 6...",48.166667,0.372678,36.000000,0.000000,15.166667,0.372678,6.000000,0.000000,976.000000,12.569805,38.333333,1.699673


orig_fid          int64
nlayers           int64
SOL_ZMX           int64
SOL_Z1            int64
SOL_Z2          float64
SOL_Z3          float64
SOL_Z4          float64
geometry       geometry
mean_1_sand     float64
std_1_sand      float64
mean_1_silt     float64
std_1_silt      float64
mean_1_clay     float64
std_1_clay      float64
mean_1_rock     float64
std_1_rock      float64
mean_1_bd       float64
std_1_bd        float64
mean_1_soc      float64
std_1_soc       float64
dtype: object

In [12]:
eesti_soil_red1_validatesoil = gpd.read_file("../data_deposit/EstSoil-EH_v1.0.shp", encoding='utf-8')


columns_to_keep = ["orig_fid",
                   'SOL_CLAY1',
                   'SOL_SILT1',
                   'SOL_SAND1',
                   'SOL_ROCK1',
                    'SOL_BD1',
                   'SOL_SOC1']

eesti_soil_red1_validatesoil_phys = eesti_soil_red1_validatesoil[columns_to_keep].copy()
del(eesti_soil_red1_validatesoil)

layer1 = pd.merge(left=layer1, right=eesti_soil_red1_validatesoil_phys, left_on='orig_fid', right_on='orig_fid', how='left')

display(layer1.sample(10))
display(layer1.dtypes)

,orig_fid,nlayers,SOL_ZMX,SOL_Z1,SOL_Z2,SOL_Z3,SOL_Z4,geometry,mean_1_sand,std_1_sand,...,mean_1_bd,std_1_bd,mean_1_soc,std_1_soc,SOL_CLAY1,SOL_SILT1,SOL_SAND1,SOL_ROCK1,SOL_BD1,SOL_SOC1
703154,702756,1,1000,1000,0.0,0.0,0.0,"POLYGON ((655960.180 6437337.560, 655956.430 6...",53.333333,0.471405,...,876.333333,16.499158,43.333333,0.942809,60,20,20,0,0.427006,28.903651
536564,528833,1,1000,1000,0.0,0.0,0.0,"POLYGON ((410245.090 6525553.420, 410246.030 6...",62.250000,1.785357,...,938.750000,18.779976,44.250000,1.089725,7,3,90,0,0.829039,9.952505
78155,86080,1,1000,1000,0.0,0.0,0.0,"POLYGON ((649357.190 6402079.620, 649349.524 6...",53.666667,0.942809,...,868.666667,38.655171,35.666667,0.471405,15,30,55,0,1.165002,4.147894
534955,527224,1,1500,1500,0.0,0.0,0.0,"POLYGON ((595555.080 6601454.920, 595551.240 6...",58.000000,3.585686,...,866.071429,28.516823,43.928571,2.282364,5,5,90,0,0.905179,8.259377
570212,570201,1,1000,1000,0.0,0.0,0.0,"POLYGON ((685502.960 6433892.210, 685502.530 6...",47.600000,0.489898,...,977.800000,24.838680,35.000000,1.095445,15,20,65,15,1.258921,3.079303
232908,232908,1,1000,1000,0.0,0.0,0.0,"POLYGON ((601283.510 6523546.720, 601276.410 6...",44.500000,0.500000,...,826.500000,24.500000,30.500000,0.500000,15,30,55,85,0.897737,8.412204
221858,221858,1,1000,1000,0.0,0.0,0.0,"POLYGON ((571619.860 6555051.800, 571613.319 6...",49.500000,0.500000,...,968.000000,15.000000,27.500000,1.500000,9,9,82,25,1.022585,6.142776
365920,365913,2,1000,650,1000.0,0.0,0.0,"POLYGON ((580682.720 6499398.160, 580686.070 6...",58.333333,1.154701,...,854.888889,27.966029,46.111111,1.099944,15,20,65,0,1.014851,6.267144
416837,403216,1,1500,1500,0.0,0.0,0.0,"POLYGON ((672013.540 6498747.100, 672014.640 6...",61.833333,1.863390,...,838.666667,8.209074,48.666667,3.423773,7,3,90,0,0.982394,6.810401
42907,42902,1,1000,1000,0.0,0.0,0.0,"POLYGON ((634828.260 6426799.200, 634831.790 6...",43.500000,0.500000,...,893.333333,10.903618,29.000000,1.000000,9,9,82,0,1.167802,4.113547


orig_fid          int64
nlayers           int64
SOL_ZMX           int64
SOL_Z1            int64
SOL_Z2          float64
SOL_Z3          float64
SOL_Z4          float64
geometry       geometry
mean_1_sand     float64
std_1_sand      float64
mean_1_silt     float64
std_1_silt      float64
mean_1_clay     float64
std_1_clay      float64
mean_1_rock     float64
std_1_rock      float64
mean_1_bd       float64
std_1_bd        float64
mean_1_soc      float64
std_1_soc       float64
SOL_CLAY1         int64
SOL_SILT1         int64
SOL_SAND1         int64
SOL_ROCK1         int64
SOL_BD1         float64
SOL_SOC1        float64
dtype: object

In [22]:
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from sklearn import datasets, linear_model
import math

for i in ['SOL_CLAY1',
           'SOL_SILT1',
           'SOL_SAND1',
           'SOL_ROCK1',
            'SOL_BD1',
           'SOL_SOC1']:
    layer1.rename( columns = { i : i.lower()}, inplace=True)

for layer in comp_l1.keys():
    print('----------------------')
    print(layer)
    # display(layer1[f"sol_{layer}1"].isnull().sum())
    # display(layer1["mean_1_" + str(layer)].isnull().sum())
    layerT = layer1.dropna()
    layerT["diff_1_" + str(layer)] = layerT[f"sol_{layer}1"] - layerT["mean_1_" + str(layer)]
    layerT["abs_diff_1_" + str(layer)] = layerT["diff_1_" + str(layer)].apply(abs)
    display(layerT["abs_diff_1_" + str(layer)].describe())
    display(layerT["std_1_" + str(layer)].describe())
    # Calculation of Mean Squared Error (MSE) 
    # RMSE = mean_squared_error(layerT[f"sol_{layer}1"].to_numpy(), layerT["mean_1_" + str(layer)].to_numpy(), squared=False)
    MSE = mean_squared_error(layerT[f"sol_{layer}1"].to_numpy(), layerT["mean_1_" + str(layer)].to_numpy())
    r2 = r2_score( layerT[f"sol_{layer}1"].to_numpy(), layerT["mean_1_" + str(layer)].to_numpy() )
    print(f"RMSE: {math.sqrt(MSE)}  MSE: {MSE}   R2:{r2}")
    print(layer)
    print('#################')
    

# mean_1_sand     float64
# std_1_sand      float64
# SOL_SAND1

# mean_1_silt     float64
# std_1_silt      float64
# SOL_SILT1

# mean_1_clay     float64
# std_1_clay      float64
# SOL_CLAY1

# mean_1_rock     float64
# std_1_rock      float64
# SOL_ROCK1 

# mean_1_bd       float64
# std_1_bd        float64
# SOL_BD1

# mean_1_soc      float64
# std_1_soc       float64
# SOL_SOC1  




----------------------
sand


C:\dev\conda3\envs\geopy2019\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\dev\conda3\envs\geopy2019\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


count    743803.000000
mean         25.110364
std          12.477622
min           0.000000
25%          16.138410
50%          28.000000
75%          34.000000
max          68.000000
Name: abs_diff_1_sand, dtype: float64

count    743803.000000
mean          1.315416
std           1.026836
min           0.000000
25%           0.500000
50%           1.200000
75%           1.892969
max          13.000000
Name: std_1_sand, dtype: float64

RMSE: 28.039637216569886  MSE: 786.2212552368511   R2:-0.29180768299342597
sand
#################
----------------------
silt


...

RMSE: 879.4352421211725  MSE: 773406.3450847253   R2:-14185571.920734271
bd
#################
----------------------
soc


C:\dev\conda3\envs\geopy2019\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\dev\conda3\envs\geopy2019\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


count    743803.000000
mean         30.265397
std           8.472439
min           0.003309
25%          26.374721
50%          31.292323
75%          35.917910
max          65.994708
Name: abs_diff_1_soc, dtype: float64

count    743803.000000
mean          1.366711
std           1.094312
min           0.000000
25%           0.500000
50%           1.224745
75%           1.986111
max          15.478658
Name: std_1_soc, dtype: float64

RMSE: 31.428909966548304  MSE: 987.7763816853992   R2:-18.12815586455163
soc
#################


In [23]:
layerT["abs_diff_1_sand"].describe()

count    743803.000000
mean         25.110364
std          12.477622
min           0.000000
25%          16.138410
50%          28.000000
75%          34.000000
max          68.000000
Name: abs_diff_1_sand, dtype: float64

In [25]:
layer_B = layerT[layerT['sol_sand1'] < 100]
layer_B["abs_diff_1_sand"].describe()

count    740380.000000
mean         25.011785
std          12.415473
min           0.000000
25%          16.000000
50%          28.000000
75%          34.000000
max          65.000000
Name: abs_diff_1_sand, dtype: float64

In [30]:
display(layer1.isnull().sum())

display(layer1.loc[layer1['AWC_L7'].isnull()])

orig_fid    0
nlayers     0
SOL_ZMX     0
SOL_Z1      0
SOL_Z2      0
SOL_Z3      0
SOL_Z4      0
SOL_AWC1    0
SOL_AWC2    0
SOL_AWC3    0
SOL_AWC4    0
AWC_L1      0
geometry    0
AWC_L2      0
AWC_L3      0
AWC_L4      0
AWC_L5      0
AWC_L6      0
AWC_L7      0
dtype: int64

,orig_fid,nlayers,SOL_ZMX,SOL_Z1,SOL_Z2,SOL_Z3,SOL_Z4,SOL_AWC1,SOL_AWC2,SOL_AWC3,SOL_AWC4,AWC_L1,geometry,AWC_L2,AWC_L3,AWC_L4,AWC_L5,AWC_L6,AWC_L7


In [29]:
cache_row = {}

for idx, row in layer1.loc[layer1['AWC_L7'].isnull()].iterrows():
    
    display(f"found {idx}, take {idx-2}")
    
    layer1.loc[idx,'AWC_L1'] = layer1.loc[idx-2,'AWC_L1']
    layer1.loc[idx,'AWC_L2'] = layer1.loc[idx-2,'AWC_L2']
    layer1.loc[idx,'AWC_L3'] = layer1.loc[idx-2,'AWC_L3']
    layer1.loc[idx,'AWC_L4'] = layer1.loc[idx-2,'AWC_L4']
    layer1.loc[idx,'AWC_L5'] = layer1.loc[idx-2,'AWC_L5']
    layer1.loc[idx,'AWC_L6'] = layer1.loc[idx-2,'AWC_L6']
    layer1.loc[idx,'AWC_L7'] = layer1.loc[idx-2,'AWC_L7']
    
    display(f"found {row['AWC_L1']}, put {layer1.loc[idx,'AWC_L7']}")

'found 14527, take 14525'

'found nan, put 18.333333333333332'

'found 14528, take 14526'

'found nan, put 18.5'

'found 14564, take 14562'

...

'found nan, put 18.0'

'found 716844, take 716842'

'found 20.0, put 19.0'

'found 743868, take 743866'

'found nan, put 18.0'

In [26]:
layer1.loc[427,'AWC_L1']

20.25

In [32]:
import functools
import statistics
import operator

known_depths_to_layer = [  
    (0,1),
    (50,2),
    (150,3),
    (300,4),
    (600,5),
    (1000,6),
    (2000,7)
]

def get_aggregate_awc_for_depths(layer_top, layer_bottom, SOL_AWC1, awc_gradient_values, known_depths_to_layer):
    filt = list(filter(lambda x: x[0] >= layer_top and x[0] <= layer_bottom, known_depths_to_layer))
    lays = list(map(lambda x: x[1], filt))
    
    if len(lays) <= 0:
        position = statistics.mean([layer_top, layer_bottom])
        if position > known_depths_to_layer[6][0]:
            # return [7]
            lays = [7]
        elif position < known_depths_to_layer[0][0]:
            # return [1]
            lays = [1]
        else:
            diffs = []
            for i in range(1,8):
                diffs.append( (statistics.stdev([position, known_depths_to_layer[i-1][0]]), i) )
            diffs.sort(key = operator.itemgetter(0))
            lays = [ diffs[0][1] ]

    vals = statistics.mean(list(map(lambda x: awc_gradient_values[int(x)], lays)))
    return vals


def aggregate_over_depths(row):
    
    SOL_AWC1 = row['SOL_AWC1']
    SOL_AWC2 = row['SOL_AWC2']
    SOL_AWC3 = row['SOL_AWC3']
    SOL_AWC4 = row['SOL_AWC4']
    
    nlayers = row['nlayers']
    SOL_ZMX = row['SOL_ZMX']
    SOL_Z1 = row['SOL_Z1']
    SOL_Z2 = row['SOL_Z2']
    SOL_Z3 = row['SOL_Z3']
    SOL_Z4 = row['SOL_Z4']
    
    SOL_Z2_new = row['SOL_Z2']
    SOL_Z3_new = row['SOL_Z3']
    SOL_Z4_new = row['SOL_Z4']
    
    AWC_L1 = row['AWC_L1']
    AWC_L2 = row['AWC_L2']
    AWC_L3 = row['AWC_L3']
    AWC_L4 = row['AWC_L4']
    AWC_L5 = row['AWC_L5']
    AWC_L6 = row['AWC_L6']
    AWC_L7 = row['AWC_L7']
    
    awc_gradient_values_pre = np.array([AWC_L1, AWC_L2, AWC_L3, AWC_L4, AWC_L5, AWC_L5, AWC_L6, AWC_L7])
    awc_gradient_values = awc_gradient_values_pre / 100
    
    if nlayers >= 1:
        # depth from top to bottom of 1st layer is exactly SOL_Z1
        layer_top = 0
        layer_bottom = SOL_Z1
        SOL_AWC1 = get_aggregate_awc_for_depths(layer_top, layer_bottom, SOL_AWC1, awc_gradient_values, known_depths_to_layer)
    
    if nlayers >= 2:
        layer_top = SOL_Z1
        layer_bottom = SOL_Z1 + SOL_Z2
        SOL_AWC2 = get_aggregate_awc_for_depths(layer_top, layer_bottom, SOL_AWC2, awc_gradient_values, known_depths_to_layer)
        SOL_Z2_new = layer_bottom
    
    if nlayers >= 3:
        layer_top = SOL_Z1 + SOL_Z2
        layer_bottom = SOL_Z1 + SOL_Z2 + SOL_Z3
        SOL_AWC3 = get_aggregate_awc_for_depths(layer_top, layer_bottom, SOL_AWC3, awc_gradient_values, known_depths_to_layer)
        SOL_Z3_new = layer_bottom
    
    if nlayers >= 4:
        layer_top = SOL_Z1 + SOL_Z2 + SOL_Z3
        layer_bottom = SOL_Z1 + SOL_Z2 + SOL_Z3 + SOL_Z4
        SOL_AWC4 = get_aggregate_awc_for_depths(layer_top, layer_bottom, SOL_AWC4, awc_gradient_values, known_depths_to_layer)
        SOL_Z4_new = layer_bottom
    
    return pd.Series([SOL_Z1, SOL_Z2_new, SOL_Z3_new, SOL_Z4_new, SOL_AWC1, SOL_AWC2, SOL_AWC3, SOL_AWC4])

layer1[['SOL_Z1', 'SOL_Z2', 'SOL_Z3', 'SOL_Z4',
        'SOL_AWC1', 'SOL_AWC2', 'SOL_AWC3', 'SOL_AWC4']] = layer1.apply(lambda x: aggregate_over_depths(x), axis=1)

In [35]:
display(layer1.sample(10))

,orig_fid,nlayers,SOL_ZMX,SOL_Z1,SOL_Z2,SOL_Z3,SOL_Z4,SOL_AWC1,SOL_AWC2,SOL_AWC3,SOL_AWC4,AWC_L1,geometry,AWC_L2,AWC_L3,AWC_L4,AWC_L5,AWC_L6,AWC_L7
140847,140716,1,1000,1000.0,0.0,0.0,0.0,0.182500,0.0000,0.0000,0.0,20.000000,"POLYGON ((413916.8999999985 6530713.420000002,...",20.00,18.500000,18.000000,17.500000,18.00,19.000000
277475,277470,3,1000,600.0,850.0,1000.0,0.0,0.184500,0.1750,0.1725,0.0,20.750000,"POLYGON ((630168.4299999997 6585855.620000001,...",20.25,19.000000,18.000000,17.500000,17.25,18.000000
80312,72027,1,1000,1000.0,0.0,0.0,0.0,0.190833,0.0000,0.0000,0.0,21.000000,"POLYGON ((636232.3200000003 6394330.5, 636228....",20.50,19.500000,19.000000,18.500000,18.50,19.000000
79983,71698,1,1000,1000.0,0.0,0.0,0.0,0.180000,0.0000,0.0000,0.0,21.000000,"POLYGON ((659092.1700000018 6442183.629999999,...",21.00,19.500000,18.000000,16.500000,16.50,17.500000
54598,53733,1,1000,1000.0,0.0,0.0,0.0,0.188333,0.0000,0.0000,0.0,20.500000,"POLYGON ((612974.5300000012 6424946.640000001,...",20.50,19.500000,19.000000,18.000000,18.00,19.000000
631319,631173,1,1000,1000.0,0.0,0.0,0.0,0.200000,0.0000,0.0000,0.0,22.000000,"POLYGON ((687400.0300000012 6456212.300000001,...",22.00,21.000000,20.000000,19.000000,19.00,19.000000
525958,518227,1,1000,1000.0,0.0,0.0,0.0,0.185833,0.0000,0.0000,0.0,20.500000,"POLYGON ((673454.244599998 6438339.107799999, ...",20.00,19.000000,18.500000,18.000000,18.00,18.500000
266664,266659,2,1000,700.0,1000.0,0.0,0.0,0.193667,0.1850,0.0000,0.0,21.333333,"POLYGON ((596014.549999997 6504163.530000001, ...",21.00,19.833333,19.333333,18.333333,18.50,18.833333
685931,685774,2,1000,600.0,1000.0,0.0,0.0,0.182500,0.1775,0.0000,0.0,19.500000,"POLYGON ((389029.0700000003 6458870.82, 389011...",19.25,18.500000,17.500000,18.000000,17.50,17.750000
267256,267251,1,1000,1000.0,0.0,0.0,0.0,0.179583,0.0000,0.0000,0.0,20.000000,"POLYGON ((405315.3100000024 6489853.510000002,...",19.25,18.750000,18.250000,17.250000,17.00,18.250000


In [36]:
layer1.to_file(f"data/eesti_soil_red1_texture_fix_geo_redo_awc_merged_layers.shp", encoding='utf-8')

C:\dev\conda3\envs\geopython-environment\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [34]:

display(layer1.isnull().sum())

orig_fid    0
nlayers     0
SOL_ZMX     0
SOL_Z1      0
SOL_Z2      0
SOL_Z3      0
SOL_Z4      0
SOL_AWC1    0
SOL_AWC2    0
SOL_AWC3    0
SOL_AWC4    0
AWC_L1      0
geometry    0
AWC_L2      0
AWC_L3      0
AWC_L4      0
AWC_L5      0
AWC_L6      0
AWC_L7      0
dtype: int64

In [19]:
import math

math.sqrt(4)

2.0